# MEMORIA


## 1. Introducción

En este proyecto se estudiará como funciona el reconocimiento de escritura. Esto es algo que lleva ya tiempo utilizandose en muchas tecnologias, como por ejemplo en las tablets. Esto reconoce la letra del usuario, y la transforma a letra digital, ahorrando el paso de teclear letra por letra en el teclado.

El objetivo de este proyecto será crear un modelo a partir de registros de letras escritas a mano, que transforme esas letras a letras digitales, y de ahi pasar a palabras.

## 2. Dataset

Para el entrenamiento de este modelo, se usara un dataset ontenido en kaggle https://www.kaggle.com/datasets/hectorherrera19/letras-alfabeto-espaol , el cual tiene una estructura de imagenes de todas las letras de alfabeto, tanto mayúsculas como minúsculas.

El dataset tiene dos estructuras, en una todas las imagenes estan ya separadas en train, test y validación, la segunda nos clasifica por carpetas todas las letras, y dentro de cada una separa las mayúsculas de las minúsculas.

Para nuestro caso usaremos la primera configuración. 

## 3. Preprocesamiento de los datos.

### 3.1 Primeros pasos

El dataset dista de una grancantidad de imagenes, de letras tanto mayusculas como minusculas, de unas demensiones de 350x350 pixeles cada una.

Inicialmente lo primero que se ha hecho con estas imagenes ha sido reescalarlas a unas dimensiones de 28x28 pixeles, y se han guardado dentro de un array, todo este proceso de ha llevado a acabo mediante una función (mirar functions.py). Posteriormente se ha comvertido este array (x_train), en un mumpy array.

Para el mejor funcionamiento del modelo, una vez hecho lo anterior, se normaliza el array con las imagenes, dividiendolo entre 255, correspondiente al blanco.

Seguidamnete se ha hecho un estudio de la cantidad de clases a clasificar. En este caso son 52, es decir, todo el alfabeto tanto en mayuscula como en minuscula a escepcion de la letra ñ. Una vez determinado esto, se ha construido un array compuesto por arrays que segun su composicion, hace referencia a una de las clases (train_y). Nuevamente se ha convertido a un numpy array y esta vez se le ha especificado que el tipo de dato sea de 32 bytes ya que no podia guardar tal cantidad de datos en memoria.



### 3.2 Aumento de datos

Tras varias pruebas, con los datos que tenemos disponibles en el dataset, se observa que la precision no es del todo buena, en torno al 0.7. Debido a esto se le aplican una serie de transformaciones a las imagenes, como rotaciones, acercamiento, deformaciones... De tal manera que del mismo grupo de datos se obtiene datos nuevos para el entrnamiento de los modelos.

Tras entrenar el modelo con estos nuevos datos, el resultado ha sido peor aun que el anterior.



In [ ]:
'''

from tensorflow.keras.preprocessing.image import ImageDataGenerator

rango_rotacion = 30
mov_ancho = 0.25
mov_alto = 0.25
rango_acercamiento=[0.5,1.5]

datagen = ImageDataGenerator(
    rotation_range = rango_rotacion,
    width_shift_range = mov_ancho,
    height_shift_range = mov_alto,
    zoom_range=rango_acercamiento,
    
)

datagen.fit(x_data_train)

'''

### 3.3 Correcciones

Tras analizar los resultados anteriores, se repetira todo lo anterior pero esta vez redimensionando las imagenes a 32x32.

## 4. Modelado

El modelo que se trabaja, es una red neuronal convolucional.

### 4.1 Funcionamiento de las redes convoluvionales

Para explicar la configuración del modelo primero hay que saber como funciona una red neuronal convolucional, y los procesos que lleva a cabo.



![imagen](../src/resources/img/cnn_esquema.PNG)

La capa de entrada hace referencia a los pixeles de las imagenes de entrada, seguido de una capa de convolución, una de agrupación y otras ocultas, como podrian ser unas capas densas. Por último la capa de salida será las clases a las que pertenece cada imagen.

#### 4.1.1. Capa de concolución

Las imagenes son un conjunto de pixeles con un valor númerico cada uno. Para interpretar formas en una imagen nos tenemos que fijar en un conjunto de pixeles, ya que un pixel por si solo no muestra nada.
Las capas de convolucion, van cogiendo los pixeles, en matrices, cada elemento de la matriz tiene un valor asignado, que se multiplicará por el valor númerico de los pixeles correspondientes. A esta matriz se la denumina núcleo. Para hacerlo más simple, se asemeja a pasarle un filtro a las imagenes. Ya que se esta trabajando con imagenes de letras, en lo que más nos va a ayudar esto, es en encotrar ejes.


![imagen](../src/resources/img/capa_conv.PNG)

#### 4.1.2. Capa de agrupación

Esta capa tiene un funcionamiento parecido, pero esta vez la matriz que va recorriendo la imagen, se queda con el pixel de mayor tamaño, lo que se traduce en imagenes más pequeñas, y con los rasgos principales más destacados.

Como consecuencia directa de la reducción de las imagenes, la siguiente capa convolucional podrá tener más nucleos, como se muestra en la siguiente imagen:


![imagen](../src/resources/img/cnn_features.PNG)

### 4.2. Configuración del modelo utilizado

Hablando más concretamente del modelo utilizado, tras varias pruebas, la configuracion que mejor precisión ha tenido, es el siguiente:

In [ ]:
''' 
model = tf.keras.Sequential([
    layers.Input(shape=(width,height,3)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(100, activation='relu'),
    layers.Dense(num_classes, activation="softmax")
])

'''

Como se ve, la primera capa de convolución tiene 32 nucleos, de 3x3, con lo que por cada imagen sacara 32 imagenes nuevas con sus respectivos filtros aplicados. Seguido de una de agrupación. Otra de convolución más pero esta vez con el doble de filtros.

El Dropout, hace referencia a ala probabilidad que tienen las neuronas de desactivarse, de esta manera evitamos que los pesos entre neuronas se desequilibren.

Respecto a las funciones de activación, no quiero mencionar mucho, pero en este caso salvo la de la capa densa de la salida que tiene una de tipo softmax, ya que tenemos un caso multiclase, las demas son de tipo RELU. Esto es, todos los valores menores que 0, se les asigna 0, y para los positivos es f(x) = x, lo que conlleva a que el aprendizaje sea rapido devido a que su deriva tiene una pendiente mas alta.

## 5. Predicción y resultados finales.

Pese a todo lo anterior, el modelo selecionado, que mejores resultados ha tenido en sus prediciones, es el siguiente:

In [ ]:
''' 
model = tf.keras.Sequential([
    layers.Input(shape=(width,height,3)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(100, activation='relu'),
    layers.Dense(num_classes, activation="sigmoid")
])

'''

![imagen](../src/resources/img/resultado.PNG)

Se observa, que en este ejemplo sale una probabilidad de 1, en la cuarta posición, que efectivamente corresponde a la b minuscula.

## 6. Conclusiones y futuros pasos.

Con una precision del 90%, se puede decir que el modelo utilizado va por muy buen camino, pese a que todavía tiene fallos a la hora de las predicciones.Además pese a que toda la teoria apuntaba a un modelo, resulta que a nivel predicciones, otro lo hace mucho mejor.
Una vez hechas algunas pruebas más, reajustando alguno de los parametros y de las capas del modelo, el siguiente paso es que aprenda a segmentar las letras aprendidas individualmente en conjunto. Es decir, que aprenda a leer texto.